In [1]:
#Compare predictions and true labels

import pandas as pd
import json
import os

#set file paths

#GROUND
# pred_boxes = pd.read_csv('/home/cara/oregon_critters/runs/detect/7_mc_ground_med/predict/pred_boxes_07_mc_ground_med.csv')
# true_boxes = pd.read_csv('/home/cara/oregon_critters/sampled_ds_ground_620_split.csv')
# json_loc = '/home/cara/oregon_critters/json/dataset_ground_620_adjxy.json'
# comparisons_csv_path = '/home/cara/oregon_critters/runs/detect/7_mc_ground_med/predict/pred_true_compare.csv'
# false_neg_loc = '/home/cara/oregon_critters/runs/detect/7_mc_ground_med/predict/pred_false_neg.csv'

#TRAIL -- fix
# pred_boxes = pd.read_csv('/home/cara/oregon_critters/trail/runs/detect/8_mc_trail_med/predict/pred_boxes_08_mc_trail_med.csv')
# true_boxes = pd.read_csv('/home/cara/oregon_critters/sampled_ds_trail_180_split.csv')
# json_loc = '/home/cara/oregon_critters/json/dataset_trail_180_adjxy.json'
# comparisons_csv_path = '/home/cara/oregon_critters/trail/runs/detect/8_mc_trail_med/predict/pred_true_compare.csv'
# false_neg_loc = '/home/cara/oregon_critters/trail/runs/detect/8_mc_trail_med/predict/pred_false_neg.csv'

#BOTH on GROUND
# pred_boxes = pd.read_csv('/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_ground/pred_boxes_09_mc_med_on_ground.csv')
# true_boxes = pd.read_csv('/home/cara/oregon_critters/sampled_ds_ground_620_split.csv')
# json_loc = '/home/cara/oregon_critters/json/dataset_bothB_adjxy.json'
# comparisons_csv_path = '/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_ground/pred_true_compare.csv'
# false_neg_loc = '/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_ground/pred_false_neg.csv'

#BOTH on TRAIL - fix
# pred_boxes = pd.read_csv('/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_trail/pred_boxes_09_mc_med_on_trail.csv')
# true_boxes = pd.read_csv('/home/cara/oregon_critters/sampled_ds_trail_180_split.csv')
# json_loc = '/home/cara/oregon_critters/json/dataset_bothB_adjxy.json'
# comparisons_csv_path = '/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_trail/pred_true_compare.csv'
# false_neg_loc = '/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_trail/pred_false_neg.csv'

#BOTH on BOTH
pred_boxes = pd.read_csv('/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_both/pred_boxes_09_mc_med_on_both.csv')
true_boxes = pd.read_csv('/home/cara/oregon_critters/sampled_ds_bothB_split.csv')
json_loc = '/home/cara/oregon_critters/json/dataset_bothB_adjxy.json'
comparisons_csv_path = '/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_both/pred_true_compare.csv'
false_neg_loc = '/home/cara/oregon_critters/both/runs/detect/9_mc_both_med/predict_on_both/pred_false_neg.csv'



In [2]:
#read in true bounding boxes, keep only columns we need, and remove '.JPG' from image name
true_df = true_boxes[['full_path_new','CName', 'X','Y','W','H', 'cam_type', 'day_night', 'project', 'group']]
true_df['full_path_new'] = true_df['full_path_new'].apply(lambda path: os.path.splitext(os.path.basename(path))[0])
#true_df.drop(columns=['full_path_new'], inplace=True)

#rename columns
true_df.rename(columns={"full_path_new": "image_name","CName": "class_true", "X":"x", "Y":"y", "W":"w", "H":"h"}, inplace=True)
true_df.head()

/tmp/ipykernel_1428530/2005514950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df['full_path_new'] = true_df['full_path_new'].apply(lambda path: os.path.splitext(os.path.basename(path))[0])
/tmp/ipykernel_1428530/2005514950.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df.rename(columns={"full_path_new": "image_name","CName": "class_true", "X":"x", "Y":"y", "W":"w", "H":"h"}, inplace=True)


,image_name,class_true,x,y,w,h,cam_type,day_night,project,group
0,10316-5__2019-08-05__04-43-55(1)_ho,Bat,1414.630021,1792.900518,824.016913,425.915281,NaN,night,COA_2019,train
1,10831-1__2019-08-07__01-08-48(7)_ho,Bat,107.568710,835.605219,564.228330,486.760322,NaN,night,COA_2019,train
2,8749-3__2019-07-19__00-53-50(1)_ho,Bat,972.177590,1817.238534,718.477801,344.788561,NaN,night,COA_2019,train
3,8749-3__2019-07-19__00-53-51(2)_ho,Bat,740.803383,774.760179,438.393235,263.661841,NaN,night,COA_2019,train
4,8749-3__2019-07-19__04-01-21(1)_ho,Bat,489.133192,1947.041287,422.156448,377.239249,NaN,night,COA_2019,train


In [3]:
#Need to add image height and width, convert from normalized, convert to COCO

#load json and turn into dataframe
with open(json_loc, 'r') as json_file:
    json_data = json.load(json_file)    

json_df = pd.DataFrame(json_data['images'])

#extract just image name from the filename
json_df['image_name'] = json_df['file_name'].str.extract(r'([^/]+)\.JPG')

#merge with pred_boxes
pred_boxes = pd.merge(pred_boxes, json_df[['image_name', 'width', 'height']], left_on='image_name', right_on='image_name', how='left')

#convert box dimensions from normalized 
pred_boxes['x_pixel'] = pred_boxes['x'] * pred_boxes['width']
pred_boxes['y_pixel'] = pred_boxes['y'] * pred_boxes['height']
pred_boxes['w_pixel'] = pred_boxes['w'] * pred_boxes['width']
pred_boxes['h_pixel'] = pred_boxes['h'] * pred_boxes['height']

#now convert from center of box to top left
pred_boxes['x_coco'] = pred_boxes['x_pixel'] - (pred_boxes['w_pixel'] / 2)
pred_boxes['y_coco'] = pred_boxes['y_pixel'] - (pred_boxes['h_pixel'] / 2)

pred_boxes.head()

,Unnamed: 0,image_name,class_ID,x,y,w,h,confidence,class_pred,width,height,x_pixel,y_pixel,w_pixel,h_pixel,x_coco,y_coco
0,0,10171-4__2019-08-12__14-30-39(1)_hy,35,0.529966,0.597495,0.042828,0.125231,0.879189,StellersJay,3840.0,2880.0,2035.06944,1720.78560,164.457984,360.665280,1952.840448,1540.452960
1,1,10171-4__2019-08-12__14-30-40(2)_hy,35,0.625867,0.572841,0.055557,0.097385,0.695314,StellersJay,3840.0,2880.0,2403.32928,1649.78208,213.338496,280.469376,2296.660032,1509.547392
2,2,10171-4__2019-08-12__14-30-40(2)_hy,11,0.230125,0.807016,0.088145,0.113618,0.455378,Grouse,3840.0,2880.0,883.68000,2324.20608,338.474880,327.219840,714.442560,2160.596160
3,3,10171-4__2019-08-12__14-30-41(3)_hy,35,0.671296,0.585399,0.054064,0.111391,0.643460,StellersJay,3840.0,2880.0,2577.77664,1685.94912,207.604224,320.806080,2473.974528,1525.546080
4,4,10171-4__2019-08-12__14-30-41(3)_hy,11,0.234348,0.805796,0.085507,0.113960,0.566589,Grouse,3840.0,2880.0,899.89632,2320.69248,328.347648,328.204800,735.722496,2156.590080


In [4]:
# Define function to calculate IoU of two bounding boxes (from Jess):

def calculate_iou(x1, y1, w1, h1, x2, y2, w2, h2):

    # Define bboxes
    box1 = [x1, y1, w1, h1]
    box2 = [x2, y2, w2, h2]

    # Convert bounding box coordinates to the format (x1, y1, x2, y2)
    box1 = [box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]]
    box2 = [box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]]

    # Calculate the coordinates of the intersection rectangle
    x_intersection = max(box1[0], box2[0])
    y_intersection = max(box1[1], box2[1])
    w_intersection = min(box1[2], box2[2]) - x_intersection
    h_intersection = min(box1[3], box2[3]) - y_intersection

    # Calculate the area of intersection
    area_intersection = max(0, w_intersection) * max(0, h_intersection)

    # Calculate the area of the union
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate IoU
    iou = area_intersection / area_union

    return iou 


In [ ]:
# Create dataframe to compare all boxes

# box_comparisons = pd.DataFrame(columns=['pred_box', 'pred_image', 'pred_class', 'confidence', 
#                                         'match', 'true_box', 'true_class', 'iou'])
# box_comparisons.head()

In [5]:
# Now iterate through every combination of predicted/true bounding boxes, calculate IoU, and find matches

thresh = 0.5 #set threshold for IoU

results_list = []

for pred_index, pred_row in pred_boxes.iterrows(): #iterate over predicted boxes

    pred_image_name = pred_row['image_name']
    matching_rows = true_df[true_df['image_name'] == pred_image_name] #find boxes in matching image
    
    # print(pred_image_name)
    # print(matching_rows)

    for true_index, true_row in matching_rows.iterrows(): #iterate over true boxes on same image

        #call function to calculate IoU 
        iou_result = calculate_iou(pred_row['x_coco'], pred_row['y_coco'], pred_row['w_pixel'], pred_row['h_pixel'],
                                   true_row['x'], true_row['y'], true_row['w'], true_row['h'])
        
        if iou_result > thresh:
            match = True
            true_class = true_row['class_true']
            
        else: 
            match = False
            true_class = 'background'

        #append a row for this predicted box
        results_list.append({'pred_box': pred_index, 
                             'pred_image': pred_image_name,
                             'pred_class': pred_row['class_pred'],
                             'confidence': pred_row['confidence'], 
                             'match': match, 
                             'true_box': true_index, 
                             'true_class': true_class, 
                             'iou': iou_result})
        
    #break

#print(results_list)

# Create dataframe from the results list
results_df = pd.DataFrame(results_list)

results_df.head()


,pred_box,pred_image,pred_class,confidence,match,true_box,true_class,iou
0,0,10171-4__2019-08-12__14-30-39(1)_hy,StellersJay,0.879189,True,46,Woodpecker,0.895872
1,1,10171-4__2019-08-12__14-30-40(2)_hy,StellersJay,0.695314,True,47,Woodpecker,0.654797
2,2,10171-4__2019-08-12__14-30-40(2)_hy,Grouse,0.455378,False,47,background,0.000000
3,3,10171-4__2019-08-12__14-30-41(3)_hy,StellersJay,0.643460,True,48,Woodpecker,0.907242
4,4,10171-4__2019-08-12__14-30-41(3)_hy,Grouse,0.566589,False,48,background,0.000000


In [6]:
# now compare classes for matches
results_df['class_match'] = results_df['pred_class'] == results_df['true_class']
results_df.head()

# add metadata back
results_df_merged = results_df.merge(true_df[['image_name', 'cam_type', 'project', 'day_night', 'group']], left_on = 'pred_image', right_on = 'image_name', how = 'left')

# Save
results_df_merged.to_csv(comparisons_csv_path)


In [7]:
#Now get false negatives by extracting indices of all true boxes that weren't matched

false_neg = true_df.query('index not in @results_df.true_box')
false_neg.head(11)

# Save
false_neg.to_csv(false_neg_loc)
